# Motion estimation

In this section we will go through how to perform motion analysis on MPS data. The material here is mostly based on the documentation at https://computationalphysiology.github.io/mps_motion

First we need to import `mps` for reading the data and `mps_motion` for performing motion analysis

In [ ]:
from pathlib import Path # For working with file paths
import mps  # For reading MPS data
import mps_motion # For motion estimation
import matplotlib.pyplot as plt # For plotting

## Loading data

Now load the data 

In [ ]:
motion_data_path = Path("motion_data.npy")

def download_data():
    print("Downloading data. Please wait...")
    link = "https://www.dropbox.com/s/kxnskaq9t0jhf88/motion_data.npy?dl=1"
    import urllib.request
    import time

    urllib.request.urlretrieve(link, motion_data_path)
    time.sleep(1.0)
    print("Done downloading data")
    
if not motion_data_path.is_file():
    download_data()
    
data = mps.MPS(motion_data_path)

Let us first print some info about the data

In [ ]:
data.info

We can also display convert the frames to a video file that we can play in the notebook

In [ ]:
movie_path = "motion_data.mp4"
mps.utils.frames2mp4(data.frames.T, movie_path, framerate=data.framerate)

In [ ]:
from IPython.display import Video
Video(movie_path)

## Creating optical flow object

Now, we will create an optical flow object which is the object we use to run the motion tracking software. Here we have chosen the Farneback optical flow algorithm

In [ ]:
opt_flow = mps_motion.OpticalFlow(data, flow_algorithm="farneback")

To list available optical flow algorithms you can use

In [ ]:
mps_motion.list_optical_flow_algorithm()

## Estimating a reference frame

Before we can run the motion analysis we need to estimate a suitable reference frame. We can do this by first estimate the velocity (let us use a spacing of 5)

In [ ]:
v = opt_flow.get_velocities(spacing=5)
print(v)

Let us compute the norm and use an algorithm for estimated the reference frame. This algorithm will use the the zero velocity baseline a find a frame where the velocity is zero. We must also provide the time stamps with the same length as the velocity trace

In [ ]:
v_norm = v.norm().mean().compute()
reference_frame_index = (
    mps_motion.motion_tracking.estimate_referece_image_from_velocity(
        t=data.time_stamps[:-5],
        v=v_norm,
    )
)
reference_frame = data.time_stamps[reference_frame_index]
print(f"Found reference frame at index {reference_frame_index} and time {reference_frame:.2f}")

Let us also plot the velocity trace and mark the point where the reference frame is chosen

In [ ]:
fig, ax = plt.subplots()
ax.plot(data.time_stamps[:-5], v_norm)
ax.plot([reference_frame], [v_norm[reference_frame_index]], "ro")
plt.show()

We can now run the optical flow algorithm to extract the displacements.

In [ ]:
u = opt_flow.get_displacements(reference_frame=reference_frame)
print(u)

We see that the object we get back is a `VectorFrameSequence`. This is a special object that represents a vector field for each image in the sequence of images, and we see that is has dimension (number of pixels in $\times$ number of pixels in $\times$ number of time steps 
 2) where the final two dimensions are the 
 and 
 component of the vectors. If we take the norm of this VectorFrameSequence we get a FrameSequence